In [58]:
{-# LANGUAGE OverloadedStrings #-}
import qualified Data.Csv as C
import qualified Data.Csv.Streaming as CS
import qualified Data.Vector as V
import qualified Data.ByteString.Lazy.UTF8 as UL
import qualified Data.ByteString.Lazy as BL
import qualified Data.ByteString as B
import Data.Either ( isLeft )
import Data.Char ( ord )
import qualified Data.Text as T
import qualified Data.Text.Encoding as E
import Control.Arrow ( (***), (&&&), first, second )

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:

In [47]:
adjust_expand :: (a -> [a]) -> Int -> [a] -> [[a]]
adjust_expand f 0 (a:t) = [b : t | b <- f a]
adjust_expand f i [] = error "Index past list"
adjust_expand f i (a:t) | i > 0 = [ a : t' | t' <- adjust_expand f (i - 1) t ]
                        | otherwise = error "Negative index"

list_or :: [a] -> [a] -> [a]
list_or [] b = b
list_or a _ = a

idelete :: Int -> [a] -> [a]
idelete 0 (a:t) = t
idelete i l@(a:t) | i > 0 = a : idelete (i - 1) t
                  | otherwise = idelete (-i) (reverse l)

both f (a, b) = (f a, f b)

Line 2: Use camelCase
Found:
adjust_expand f 0 (a : t) = ...
Why not:
adjustExpand f 0 (a : t) = ...Line 8: Use camelCase
Found:
list_or [] b = ...
Why not:
listOr [] b = ...

In [38]:
f <- B.readFile "cities500.txt"

In [59]:
-- B.writeFile "cities500_resplit.txt" $ E.encodeUtf8 $ T.unlines $ map (T.intercalate "\t") $ concatMap (adjust_expand ((`list_or` [""]) . T.splitOn ",") 3 . T.splitOn "\t") $ T.lines $ E.decodeUtf8 f
(cities, names) = both T.unlines $ unzip $ map ((
            (T.intercalate "\t" . idelete 3)
            &&& (T.intercalate "\n" . (\l -> [ T.intercalate "\t" [head l, n] | n <- case l !! 3 of { "" -> [l !! 1]; s -> T.splitOn "," s } ])
                )
    ) . T.splitOn "\t") $ T.lines $ E.decodeUtf8 f
B.writeFile "cities.txt" $ E.encodeUtf8 cities
B.writeFile "names.txt" $ E.encodeUtf8 names

In [ ]:
CS.decode 